In [ ]:
!pip install names

In [1]:
import nltk
from nltk.corpus import stopwords, names
from nltk.corpus import names as nm
import string
import names
from nltk import word_tokenize, FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from nltk.stem.wordnet import WordNetLemmatizer
import random
import pandas as pd
import pickle
import numpy as np
import csv


# Import Data and Split into Training and Test Sets

In [18]:
# raw romance, horror and fantasy scripts imported
horror_scripts_raw = pickle.load(open('hor_scripts.p','rb'))

In [19]:
romance_scripts_raw = pickle.load(open('rom_scripts.p','rb'))

In [20]:
fantasy_scripts_raw = pickle.load(open('fan_scripts.p','rb'))

In [23]:
# each shuffled so that they are no longer in alphabitical order
def dict_shuffle(dict_):
    l = list(dict_.items())
    random.shuffle(l)
    dict_1 = dict(l)
    return dict_1

In [24]:
horror_scripts_raw_shuff = dict_shuffle(horror_scripts_raw)

In [25]:
romance_scripts_raw_shuff = dict_shuffle(romance_scripts_raw)

In [26]:
fantasy_scripts_raw_shuff = dict_shuffle(fantasy_scripts_raw)

In [27]:
# Romance label is 0, horror 1 and fantasy 2.
# label = {'Romance':0,'Horror':1,'Fantasy':2}

In [51]:
len(horror_scripts_raw_shuff.keys()), len(romance_scripts_raw_shuff.keys()), len(fantasy_scripts_raw_shuff.keys())

(147, 197, 116)

In [52]:
len(horror_scripts_raw_shuff['28 DAYS LATER'])

110566

In [58]:
keys_list_train = []
keys_list_test = []
X_train = []
X_test = []
y_train = []
y_test = []

In [59]:
# 116 scripts are used from each genere to maintain sample balance
# the data is split into 80% training and 20% testing
def train_test_split(data, feat_train, label_train, feat_test, label_test, label):
    counter = 0
    for k,v in data.items():
        if counter < 92:
            print('train',k,counter);
            feat_train.append(v)
            keys_list_train.append(k)
            label_train.append(label)
            counter +=1
        elif counter >= 92 and counter < 116:
            print('test',k,counter);
            feat_test.append(v)
            keys_list_test.append(k)
            label_test.append(label)
            counter +=1
        else:
            break

In [60]:
train_test_split(romance_scripts_raw_shuff,X_train,y_train,X_test,y_test,0)
train_test_split(horror_scripts_raw_shuff,X_train,y_train,X_test,y_test,1)
train_test_split(fantasy_scripts_raw_shuff,X_train,y_train,X_test,y_test,2)

train THE LAST OF THE MOHICANS 0
train "WHITE CHRISTMAS" 1
train THE BIJOU 2
train Crouching Tiger, Hidden Dragon 3
train MARLEY & ME 4
train Chasing Amy 5
train "Never Been Kissed" 6
train THE ARTIST 7
train THE GREAT GATSBY 8
train AIRPLANE 9
train HER 10
train JUNO 11
train "IT'S A WONDERFUL LIFE" 12
train T H E   A M E R I C A N   P R E S I D E N T 13
train MRS BROWN BY JEREMY BROCK 14
train W I L D  A T  H E A R T 15
train "BIRTHDAY GIRL" 16
train PRIDE AND PREJUDICE 17
train HIGH FIDELITY 18
train VANILLA SKY 19
train PEARL HARBOR - by Randall Wallace 20
train SILVER LININGS PLAYBOOK 21
train "The Princess Bride" 22
train 500 DAYS OF SUMMER 23
train CHARADE 24
train THE THEORY OF EVERYTHING 25
train EASY "A" 26
train DATE NIGHT 27
train "THE HUDSUCKER PROXY" 28
train TALL IN THE SADDLE 29
train FADE IN 30
train M  U  L  H  O  L  L  A  N  D 31
train THE BODYGUARD 32
train HEAVENLY CREATURES 33
train AUTUMN IN NEW YORK 34
train THE BATTLE OF SHAKER HEIGHTS 35
train A WALK TO REMEMB

In [132]:
# pickle.dump(y_train,open('y_train.p','wb'))

In [133]:
# pickle.dump(y_test,open('y_test.p','wb'))

In [ ]:
# inspect data to ensure the function worked

In [63]:
len(X_train), len(y_train),len(keys_list_train)

(276, 276, 276)

In [64]:
len(X_test), len(y_test), len(keys_list_test)

(72, 72, 72)

In [65]:
X_train[239][0:40]

'SHREK Written by William Steig & Ted Ell'

# Stopwords and Tokenize

In [66]:
# list of stopwords is compiled to include
stopwords_list = stopwords.words("english") # nltk's english library
stopwords_list += string.punctuation # all punctuation
stopwords_list += ["''", '""', '...', '`']
stopwords_list += [str(num) for num in range(0,10)] # numbers from 0-9
stopwords_list += list(map(lambda x: x.lower(), nm.words())) # male and female first names from nltk library

##### With Lemme

In [67]:
lemmatizer = WordNetLemmatizer()
def process_script_w_lemme(script):
    pattern = "([a-zA-Z]+(?:'[a-z]*)?)"
    script_tokens_raw =  nltk.regexp_tokenize(script, pattern) # tokenize while elemeninating ' suffix
    script_tokens = [word.lower() for word in script_tokens_raw] # lower case every word
    # lemmatize based with pos set to verb, noun and adjective.
    script_words_stopped = [lemmatizer.lemmatize(word) for word in script_tokens if word not in stopwords_list]
    script_words_stopped = [lemmatizer.lemmatize(word,pos='a') for word in script_words_stopped if word not in stopwords_list]
    script_words_stopped = [lemmatizer.lemmatize(word,pos='v') for word in script_words_stopped if word not in stopwords_list]
    return script_words_stopped

In [68]:
processed_script_train_lemme = list(map(process_script_w_lemme, X_train))
processed_script_test_lemme = list(map(process_script_w_lemme, X_test))

In [121]:
# pickle.dump(processed_script_train_lemme,open('processed_script_train_lemme.p','wb'))

In [122]:
# pickle.dump(processed_script_test_lemme,open('processed_script_test_lemme.p','wb'))

In [71]:
len(processed_script_train_lemme[0])

13351

In [128]:
len(processed_script_train_lemme), processed_script_train_lemme[2][0:3]

(276, ['bijou', 'revise', 'draft'])